In [1]:
import pandas as pd 
import numpy as np 
import random

import matplotlib.pyplot as plt # data visualization
import seaborn as sns
%matplotlib inline

# display trick to display all columns of large dataframes
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None 

In [2]:
DATA_FOLDER = 'yelp_dataset'

### Loading business data

In [3]:
business_df = pd.read_json('{}/yelp_academic_dataset_business.json'.format(DATA_FOLDER), lines=True)
print(business_df.shape)

(209393, 14)


In [4]:
# only keep the businesses that are still open in the dataset
business_df = business_df[business_df['is_open']==1]
print(business_df.shape)

(168903, 14)


In [5]:
df_explode = business_df.assign(categories = business_df.categories.str.split(', ')).explode('categories')
#df_explode.categories.value_counts()
df_explode[df_explode.categories.str.contains('Home ', case=False, na=False)].categories.value_counts()

Home Services                20653
Home & Garden                 6328
Home Decor                    2003
Home Cleaning                 1288
Home & Rental Insurance        431
Home Inspectors                356
Home Health Care               316
Home Window Tinting            227
Home Organization              194
Home Theatre Installation      188
Home Automation                187
Home Staging                    84
Home Network Installation       66
Home Energy Auditors            39
Home Developers                 32
Mobile Home Parks               30
Mobile Home Dealers             16
Mobile Home Repair              15
Name: categories, dtype: int64

In [6]:
# home services businesses
business_home = business_df[business_df['categories'].str.contains('Home Services', case=False, na=False)]
print(business_home.shape)
business_home.head()

(20653, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."
10,MbZMmwo-eL0Jnm_Yb9KJrA,Chinook Landscaping and Design,,Calgary,AB,T2J 2P2,50.945960,-114.037207,5.0,3,1,None,"Contractors, Landscaping, Home Services","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ..."
13,JjJs3o60uQCfctDjs45cmA,Convertabath,"116 N Roosevelt Ave, Bldg B, Ste 124",Chandler,AZ,85226,33.303869,-111.951660,2.5,10,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Contractors, Home Services, Local Services","{'Monday': '9:0-16:0', 'Tuesday': '9:0-16:0', ..."
64,EjRyYGHUxlwxYY8N73vv2w,"Verizon Authorized Retailer, TCC","2218 E Williams Field Rd, Ste 115",Gilbert,AZ,85295,33.309233,-111.743293,3.0,7,1,"{'BusinessParking': '{'garage': False, 'street...","Mobile Phones, Internet Service Providers, Mob...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
72,Ar7b3_R0OBrSIbRmGvhWwg,Garage Experts of Charlotte,,Charlotte,NC,28202,35.232678,-80.846082,3.5,3,1,{'BusinessAcceptsCreditCards': 'True'},"Cabinetry, Home Services, Flooring, Self Stora...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."


### Loading review data

In [7]:
review_json_path = '{}/yelp_academic_dataset_review.json'.format(DATA_FOLDER)

size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

### Merging business and review data

In [8]:
chunk_list = []
for chunk_review in review:
    # drop columns that aren't needed
    # chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_home, chunk_review, on='business_id', how='inner')
    # show feedback on progress
    print(f'{chunk_merged.shape[0]} out of {size:,} related reviews')
    chunk_list.append(chunk_merged)
# after trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

39183 out of 1,000,000 related reviews
41103 out of 1,000,000 related reviews
39556 out of 1,000,000 related reviews
44262 out of 1,000,000 related reviews
39887 out of 1,000,000 related reviews
42064 out of 1,000,000 related reviews
42410 out of 1,000,000 related reviews
44223 out of 1,000,000 related reviews
1638 out of 1,000,000 related reviews


In [9]:
csv_name = 'yelp_dataset/yelp_reviews_home_categories.csv'
df.to_csv(csv_name, index=False)